In [1]:
import pandas as pd
import os
import sqlite3 
import sqlite_utils
from lxml import etree
import unicat
from tqdm.auto import tqdm

## Get vabb_isbn, unicat_work, manifestation and failure from the unicat.db

The original unicat database contained information on the isbns from VABB, and the results of the API search.

In [3]:
con = sqlite3.connect(r"new_unicat.db")

In [4]:
unicat_work = pd.read_sql("SELECT id, url, raw_result FROM unicat_work", con)

In [5]:
vabb_isbn = pd.read_sql("SELECT number_nr, isbn FROM vabb_isbn", con)

In [6]:
failure = pd.read_sql("SELECT isbn, error FROM failure", con)

In [7]:
manifestation = pd.read_sql("SELECT isbn, work_id FROM manifestation", con)

## Create new database

We are creating a new database to store all the information from the UniCat search

In [8]:
db = sqlite_utils.Database("unicat_holdings.db")

In [9]:
db.table_names()

[]

In [10]:
db_con = sqlite3.connect("unicat_holdings.db")

### Add the data to the new database

In [11]:
unicat_work = unicat_work.set_index("id")

In [12]:
unicat_work.to_sql("unicat_work", db_con)

44229

In [13]:
vabb_isbn = vabb_isbn.set_index("number_nr")

In [14]:
vabb_isbn.to_sql("vabb_isbn", db_con)

96191

In [15]:
failure = failure.set_index("isbn")

In [16]:
failure.to_sql("failure", db_con)

1

In [17]:
db.table_names()

['unicat_work', 'vabb_isbn', 'failure']

### Get the ISBN that was used for data retrieval

In [18]:
unicat_work = db.table("unicat_work",pk="id")

In [19]:
isbn_search = db["isbn_search"]
if "isbn_search" not in db.table_names():
    isbn_search = isbn_search.create({
        "work_id": int,
        "isbn":str,
    })

In [20]:
db.table_names()

['unicat_work', 'vabb_isbn', 'failure', 'isbn_search']

In the following, we check what ISBN was used for data retrieval and store this in the "isbn_search" table.

In [21]:
for row in tqdm(unicat_work.rows_where("raw_result is not null")):
    book = unicat.Book(row["raw_result"])
    data = [
        {"work_id": row["id"], "isbn": book.isbn_look_up()}
    ]
    isbn_search.insert_all(data)

0it [00:00, ?it/s]

In [23]:
db.table_names()

['unicat_work', 'vabb_isbn', 'failure', 'isbn_search']

## Add holding information

In [24]:
# create table "holding"
holding = db["holding"]
if "holding" not in db.table_names():
    holding = holding.create({
        "work_id": int,
        "location": str,
        "uri": str,
    })

In [25]:
db.table_names()

['unicat_work', 'vabb_isbn', 'failure', 'isbn_search', 'holding']

In [26]:
# retrieve the holding information from the results of the API search in the new table "holding"
for row in tqdm(unicat_work.rows_where("raw_result is not null")):
    book = unicat.Book(row["raw_result"])
    data = [
        {"work_id": row["id"], "location": location, "uri": uri}
        for location, uri
        in book.holdings()
    ]
    holding.insert_all(data)

0it [00:00, ?it/s]

In [27]:
# get the "holding" information
df = pd.read_sql(
    """
    SELECT work_id, location
    FROM holding
    """, db_con)

In [28]:
# calculate the holding count for each work-id
holdingcounts = pd.DataFrame(df.groupby("work_id").location.nunique()).reset_index()
holdingcounts = holdingcounts.rename(columns={"location":"holdingcount"})

In [29]:
# add the holdingcount information to the database in the table "holdingcount"
holdingcounts.set_index("work_id", inplace=True)
holdingcounts.to_sql("holdingcount",db_con)

25524

In [30]:
db.table_names()

['unicat_work',
 'vabb_isbn',
 'failure',
 'isbn_search',
 'holding',
 'holdingcount']

## Add the manifestation table

In [31]:
manifestation.set_index("isbn", inplace=True)
manifestation.to_sql("manifestation", db_con)

62198

## Add the library type

In [32]:
# file manually created based on information on UniCat website and internet searches.
# contains list of unicat libraries, other names used to refer to the library and the library type
libraries = pd.read_excel("cleaning/data/list_unicat_library_type.xlsx")

In [33]:
libraries.set_index("library", inplace=True)

In [34]:
libraries.to_sql("libraries", db_con)

78

In [35]:
# The database now contains the following:
db.table_names()

['unicat_work',
 'vabb_isbn',
 'failure',
 'isbn_search',
 'holding',
 'holdingcount',
 'manifestation',
 'libraries']